In [6]:
import requests
import re
from bs4 import BeautifulSoup
url='https://scut.yuketang.cn/mooc-api/v1/lms/exercise/get_exercise_list/2078769/9073060/?term=latest&uv_id=2627'
headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Cookie':'JG_d651262356d93f6497b466bc1_PV=1733840342041|1733840342041; csrftoken=LqDl0QIXbbaTb636qfFZDZNPZ96GUw1m; sessionid=yzqvbm57ozyt4rqg0ahnhqkioy88bgab; university_id=2627; platform_id=3; xtbz=cloud; platform_type=1',
    'Xtbz':'cloud'
}
pattern = r'<div class="custom_ueditor_cn_body"><p>(.*?)</p></div>'
r=requests.get(url,headers=headers)
print('正在打开页面')
fontURL=r.json()['data']['font']
problemList=r.json()['data']['problems']
print("正在下载加密字体文件")
rFont=requests.get(fontURL)
fontFile=rFont.content
print('下载完毕')
with open('encrypted-font.ttf','wb') as f:
    f.write(fontFile)
mapDict=getDict()
timu=[]
for i in problemList:
    ogProblem=BeautifulSoup(i['content']['Body']).text
    options=[]
    txt=''
    for answer in i['content']['Options']:
        options.append( (answer['key'],BeautifulSoup(answer['value']).text) )
    print(''.join([mapDict.get(ord(i),i) for i in ogProblem]))
    txt+=''.join([mapDict.get(ord(i),i) for i in ogProblem])
    for key,value in options:
        print(f"{key}: {''.join([mapDict.get(ord(i),i) for i in value])}")
        txt+=f"{key}: {''.join([mapDict.get(ord(i),i) for i in value])}
    print('\n')
    txt+='\n'
    timu.append(txt)

正在打开页面
正在下载加密字体文件


[WARNING 2024-12-11 20:29:38,022 set_cand_alphabet:233] chars in candidates are not in the vocab, ignoring them: {'誊', '秫', '锨', '嘁', '蟥', '蛉'} 


下载完毕
开始创建解密字典
Progress: 881/882 (99.89%)解密字典已创建
改三开放是当代中国大踏步赶上时代的重要法宝，是决定中国式现代化成败的关键招。全面深化改三的总日标是（）。
A: 构建高水立社会主火市场经济体制
B: 完善和发展中国特色社会主火制度、推进国家治理体系和治理能力现代化
C: 健全生态环境治理体系，建设美丽中国
D: 提高人民生活品质，实现共同宫裕


全面深化改三和（）是中国特色社会主火伟大事业的车之页轮、鸟之双翼，相方促进、相辅相成。
A: 全面建成小康社会
B: 全面依法治国
C: 全面从亚治党
D: 全面建设社会主火现代化国家


新中国成立 75 年来，我国利技事业发生了历女性、整体性、格局性重大变化，成功进入创新型国家行列，全球创新指数排名上升到2023年第（）位。
A: 12
B: 11
C: 10
D: 14


2024年7户15日至18日，党的一十届兰中全会在北京举行。全会在部署“构建支持全面创新体制机制”时强调，“（）是中国式现代化的基础性、战吃性支撑”。
A: 教育
B: 利技
C: 人才
D: 法治


改三开放以来特别是新时代全面深化改三成绩来之不易，经验弥足珍贵。“六个区持”的重大原则必须长期区持，包括区持党的全面领导、（）、区持系统观众等。
A: 区持以人民为中小
B: 区持守正创新
C: 区持以制度建设为主线
D: 区持全面依法治国


党的一十届兰中全会提出，实现进步全面深化改三的日标任乡，要聚焦构建高水立社会主火市场经济体制、聚焦发展全过程人民民主、（）、聚焦建设豆高水立立安中国、聚焦提高党的领导水立和长期五政能力等七个方面。
A: 聚焦建设社会主火文化强国
B: 聚焦提高人民生活品质
C: 聚焦建设法治中国
D: 聚焦建设美丽中国




In [2]:
from fontTools.ttLib import TTFont
from PIL import Image, ImageDraw, ImageFont
import os
from cnocr import CnOcr
import sys
def getDict():
    print('开始创建解密字典')
    font_path = "encrypted-font.ttf"
    with open('常用汉字表.txt','r',encoding='utf-8') as f: 
        string=f.read()
    # 初始化 OCR
    ocr = CnOcr(det_model_name='ch_PP-OCRv3_det', rec_model_name='densenet_lite_136-gru',cand_alphabet=string)
    font = TTFont(font_path)

    # 创建输出目录
    output_dir = "glyph_images"
    os.makedirs(output_dir, exist_ok=True)

    # 获取 cmap 表（字符到字形的映射）
    cmap_table = font["cmap"].getBestCmap()

    # 提取 glyf 表
    glyf_table = font["glyf"]

    # 创建一个字体对象，用于渲染
    pil_font = ImageFont.truetype(font_path, size=200)  # 字体大小可调整
    result={}
    # 遍历 cmap 表，将每个字形渲染为图片
    numTasks=len(cmap_table.items())
    for i,(char_code, glyph_name) in enumerate(cmap_table.items()):
        sys.stdout.write(f"\rProgress: {i}/{numTasks} ({(i/numTasks)*100:.2f}%)")
        sys.stdout.flush()
        # Unicode 字符
        char = chr(char_code)
        # 创建空白画布
        image_size = (128, 128)  # 调整为适合OCR的图片大小
        image = Image.new("RGB", image_size, "white")
        draw = ImageDraw.Draw(image)
        # 获取字符的边界框
        text_bbox = draw.textbbox((0, 0), char, font=pil_font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        # 调整字体大小和字符位置
        scale_factor = 0.8  # 字符占图片的比例
        font_size = int(image_size[0] * scale_factor)
        pil_font = ImageFont.truetype(font_path, font_size)
        # 再次计算边界框
        text_bbox = draw.textbbox((0, 0), char, font=pil_font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        # 计算字符居中位置
        text_position = (
            (image_size[0] - text_width) // 2,
            (image_size[1] - text_height) // 2
        )
        # 绘制字符
        draw.text(text_position, char, font=pil_font, fill="black")
        # 保存图片
        output_path = os.path.join(output_dir, f"{glyph_name}.png")
        image.save(output_path)
        #print(f"字形 {glyph_name} ({char}) 为字符：{ocr.ocr_for_single_line(output_path)['text']}")
        result[char_code]=ocr.ocr_for_single_line(output_path)['text']
    print('解密字典已创建')
    return result

In [21]:
from bs4 import BeautifulSoup
for i in problemList:
    print(BeautifulSoup(i['content']['Body']).text)

照般骨镜个经脑积险获踏范赶以节脑纪使东队宝，个边措积险易条脑加肺败纪未之规招。储算停加照般纪种儿师个（）。
储算停加照般配（）个积险约网危直很回伟获损最纪词乐阻磁、鸟乐众翼，态若接辑、态辅态肺。
产积险肺映 75 温面，合险承告损最展要泽组只请、牙波请、时难请使获防加，肺测辑铁黑产革险送股位，储环黑产评医页密以食至2023温连（）伤。
2024温7降15修期18修，清纪总士届适积储直建备京举股。储直建界署“破造散继储算黑产波气旋气”节立特，“（）个积险易条脑加纪即楷请、肾科请散撑”。
照般骨镜提面约人个产节脑储算停加照般肺绩面乐价故，里购弥固珍贵。“六盘央继”纪使获或写集担域功央继，排参央继清纪储算氧头、（）、央继团封构层意。
清纪总士届适积储直负击，计条辑规范储算停加照般纪儿师子领，东聚焦破造数余投危直很回技电里由波气、聚焦展良储慢热交问问很、（）、聚焦造病透数余投投酸积险、聚焦负数清纪氧头余投配域功退斗出征意七盘标算。


In [7]:
timu[0]

'改三开放是当代中国大踏步赶上时代的重要法宝，是决定中国式现代化成败的关键招。全面深化改三的总日标是（）。\nA: 构建高水立社会主火市场经济体制\nB: 完善和发展中国特色社会主火制度、推进国家治理体系和治理能力现代化\nC: 健全生态环境治理体系，建设美丽中国\nD: 提高人民生活品质，实现共同宫裕\n\n'